In [ ]:
import os
import re
import csv
import json
import time
import datetime
import jsonlines
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from pandas import Series
from collections import Counter
import matplotlib.pyplot as plt
from scipy.stats import f_oneway
from datetime import timedelta

sns.set(style="whitegrid")

csv_file = r'E:\Documentos\Jorge\University\Master\Semester 1\ECMM443 Introduction to Data Science\Tweets\tweets.csv'
working_directory = r'D:\Documentos\Jorge\University\Master\Semester 1\Tweets\geoEurope'



tweets_missed = []

In [ ]:
tweets_missed = []
header = ['tweet_id', 'timestamp', 'user_id', 'user_name', 'full_text', 'mentions_users_ids', 'mentions_users_names', 'country_code', 'coordinate_lon', 'coordinate_lat', 'point1_lon', 'point1_lat', 'point2_lon', 'point2_lat', 'point3_lon', 'point3_lat', 'point4_lon', 'point4_lat']

start = time.time()
with open(csv_file, 'w', encoding='UTF8', newline='') as c:
    x = 1
    y = 1
    writer = csv.DictWriter(c, fieldnames=header)
    c.truncate()
    writer.writeheader()
    for file_name in [file for file in os.listdir(working_directory) if file.endswith('.json')]:
        with jsonlines.open(working_directory + '\\' + file_name, 'r') as f:
            print('Started: ' + str(x) + ' ' + file_name)
            x+=1
            for each in f:
                y = y + 1
                tweet = {}
                mentions_users_ids = []
                mentions_users_names = []
                if 'created_at' in each:
                    tweet['tweet_id'] = each['id']
                    tweet['timestamp'] = int(each['timestamp_ms'])
                    tweet['user_id'] = each['user']['id']
                    tweet['user_name'] = each['user']['screen_name']
                    if 'extended_tweet' in each:
                        if 'full_text' in each['extended_tweet']:
                            if each['extended_tweet']['full_text'] is not None: 
                                tweet['full_text'] = repr(each['extended_tweet']['full_text'])
                        if 'entities' in each['extended_tweet']:
                            if 'user_mentions' in each['extended_tweet']['entities']:
                                if each['extended_tweet']['entities']['user_mentions']:
                                    if len(each['extended_tweet']['entities']['user_mentions']) > 1:
                                        current_ids = ''
                                        current_names = ''
                                        for i in range(0, len(each['extended_tweet']['entities']['user_mentions'])):
                                            if i == 0:
                                                mentions_ids = str(each['extended_tweet']['entities']['user_mentions'][i]['id'])
                                                current_ids = mentions_ids
                                                mentions_names = str(each['extended_tweet']['entities']['user_mentions'][i]['screen_name'])
                                                current_names = mentions_names
                                                continue
                                            mentions_ids = str(each['extended_tweet']['entities']['user_mentions'][i]['id'])
                                            current_ids = current_ids + ', ' + mentions_ids
                                            mentions_names = str(each['extended_tweet']['entities']['user_mentions'][i]['screen_name'])
                                            current_names = current_names + ', ' + mentions_names
                                        tweet['mentions_users_ids'] = current_ids
                                        tweet['mentions_users_names'] = current_names
                                    else:
                                        tweet['mentions_users_ids'] = each['extended_tweet']['entities']['user_mentions'][0]['id']
                                        tweet['mentions_users_names'] = each['extended_tweet']['entities']['user_mentions'][0]['screen_name']
                    else:
                        if each['entities']['user_mentions']:
                            if len(each['entities']['user_mentions']) > 1:
                                current_ids = ''
                                current_names = ''
                                for i in range(0, len(each['entities']['user_mentions'])):
                                    if i == 0:
                                        mentions_ids = str(each['entities']['user_mentions'][i]['id'])
                                        current_ids = mentions_ids
                                        mentions_names = str(each['entities']['user_mentions'][i]['screen_name'])
                                        current_names = mentions_names
                                        continue
                                    mentions_ids = str(each['entities']['user_mentions'][i]['id'])
                                    current_ids = current_ids + ', ' + mentions_ids
                                    mentions_names = str(each['entities']['user_mentions'][i]['screen_name'])
                                    current_names = current_names + ', ' + mentions_names
                            else:
                                tweet['mentions_users_ids'] = each['entities']['user_mentions'][0]['id']
                                tweet['mentions_users_names'] = each['entities']['user_mentions'][0]['screen_name']
                        tweet['full_text'] = repr(each['text'])
                    if each['place']:
                        if each['place']['country_code']:
                            tweet['country_code'] = str(each['place']['country_code'])
                        if each['place']['bounding_box']:
                            tweet['point1_lon'] = each['place']['bounding_box']['coordinates'][0][0][0]
                            tweet['point1_lat'] = each['place']['bounding_box']['coordinates'][0][0][1]

                            tweet['point2_lon'] = each['place']['bounding_box']['coordinates'][0][1][0]
                            tweet['point2_lat'] = each['place']['bounding_box']['coordinates'][0][1][1]

                            tweet['point3_lon'] = each['place']['bounding_box']['coordinates'][0][2][0]
                            tweet['point3_lat'] = each['place']['bounding_box']['coordinates'][0][2][1]

                            tweet['point4_lon'] = each['place']['bounding_box']['coordinates'][0][3][0]
                            tweet['point4_lat'] = each['place']['bounding_box']['coordinates'][0][3][1]
                    if each['coordinates']:
                        tweet['coordinate_lon'] = each['coordinates']['coordinates'][0]
                        tweet['coordinate_lat'] = each['coordinates']['coordinates'][1]
                    writer.writerow(tweet)
                elif 'tweets_missed':
                    tweets_missed.append(each["tweets_missed"])
                else:
                    print('other line error:', y)
                    continue 
    end = time.time()
    print(end - start)

# Part 1

In [ ]:
part1_cols = ['tweet_id', 'timestamp', 'user_id']
df = pd.read_csv(csv_file, usecols = part1_cols)

### Question 1

In [ ]:
#Total number of tweets
num_tweets = df['tweet_id'].count()
print('There are a total of', num_tweets, 'tweets present')

#Remove duplicates
df_unique = df.drop_duplicates()
df_unique = df_unique.dropna(how='all', axis=0)
print('After removing duplicate tweets, there are a total of', len(df_unique), 'unique tweets present')

#Tweets missing
print(sum(tweets_missed), 'tweets are missing')

In [ ]:
unique_users = df['user_id'].drop_duplicates()
print('There are', unique_users.count(), 'unique users')

### Question 2

In [ ]:
time_series_df = df_unique.copy()
time_series_df = time_series_df.dropna(how='all', axis=0)
time_series_df = time_series_df.reset_index(drop=True)

index_time = []
for i in range(0, len(time_series_df['timestamp'])):
    timestamp = datetime.datetime.fromtimestamp(int(time_series_df['timestamp'][i]) / 1000)
    index_time.append(timestamp)
    

time_series_df.insert(1, 'index_times', index_time)
time_series_df = time_series_df.set_index(time_series_df['index_times'])
time_series_df = time_series_df.drop('timestamp', axis=1)

def f(x):
     return Series(dict(Number_of_tweets = x['tweet_id'].count(), ))

daily_count = time_series_df.groupby(time_series_df.index.date).apply(f)
print('There are', len(daily_count), 'days in the dataset')

In [ ]:
plt.figure(figsize=(20, 10))
plt.xticks(rotation = 30, fontsize=15)
plt.yticks(fontsize=15)
plt.xlabel('Dates', labelpad=30, fontsize=20)
plt.ylabel('Number of tweets', labelpad=30, fontsize=20)
plt.title('Number of Tweets per day', fontsize=25, pad=20)
plt.plot(daily_count['Number_of_tweets'], lw=2);

### Question 3

In [ ]:
count_by_day = time_series_df['tweet_id'].groupby(by=time_series_df['index_times'].dt.date).count()

weekday = []
weekend = []

for each in range(0, len(count_by_day)):
    if count_by_day.index[each].weekday() < 5:
        weekday.append(count_by_day[each])
    else:
        weekend.append(count_by_day[each])

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
plt.setp(ax, xticklabels=['Weekday', 'Weekend'])
plt.ylabel('Number of Tweets', fontsize=14, labelpad=20)
plt.xticks(fontsize=13)plt.yticks(fontsize=13)
plt.title('Average number of tweets on weekdays VS weekends', fontsize=20, pad=20)
plt.boxplot([weekday, weekend]);

In [ ]:
#Statistcally significant
print(f_oneway(weekday, weekend))
print('There are not enough statisitcally significant differences between the number of tweets on weekdays and weekends')

### Question 4

In [ ]:
count_by_day2 = time_series_df['tweet_id'].groupby(by \
                            = [time_series_df['index_times'].dt.date, time_series_df['index_times'].dt.hour]).count()

count_hour = []
count_index = []

for each in range(0, len(count_by_day2)):
    if count_by_day2.index[each][0].weekday() < 5:
        count_hour.append(count_by_day2[each])
        count_index.append(count_by_day2.index[each][1])

In [ ]:
my_series = pd.Series(count_hour, count_index)
count_my_series = my_series.groupby(by = my_series.index).sum()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
ax.ticklabel_format(style='plain')
plt.xticks(np.arange(0,24,2), fontsize=13);
plt.yticks(fontsize=13)
plt.title('Number of Tweets per hour on Weekdays', fontsize=25, pad=20)
plt.plot(count_my_series.index, count_my_series.values);

# Part 2

In [ ]:
part2_cols = ['tweet_id', 'user_id', 'mentions_users_ids', 'country_code']
df = pd.read_csv(csv_file, usecols = part2_cols)

In [ ]:
#Remove duplicates
df.drop_duplicates(inplace=True)
df.dropna(how='all', axis=0, inplace=True)

In [ ]:
df.dropna(subset=['mentions_users_ids'], inplace=True)

for i in range(0, len(df['mentions_users_ids'].values)):
    df['mentions_users_ids'].values[i] = df['mentions_users_ids'].values[i].split(", ")

### Question 1

In [ ]:
count_tweets = df['user_id'].groupby(by = df['user_id']).count()
count_tweets = count_tweets.groupby(by=count_tweets.values).count()
df_count = pd.DataFrame(data = count_tweets)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
plt.hist(df_count, bins= 25, range = [0, 15000], log = True)
plt.xticks(np.arange(0, 15000, 1000))
plt.title('Histogram of the number of users and the tweets they make', fontsize=20)
plt.xlabel('Number of tweets', labelpad=30, fontsize=20)
plt.ylabel('Number of users (log scale)', labelpad=30, fontsize=20);

In [ ]:
fig, ax = plt.subplots(figsize=(12, 7))
plt.hist(df_count, bins= 10000)
plt.xlim(0, 250)
plt.xticks(np.arange(0, 250, 50))
plt.title('Histogram of the number of users and the tweets they make', fontsize=20)
plt.xlabel('Number of tweets', labelpad=30, fontsize=20)
plt.ylabel('Number of users', labelpad=30, fontsize=20);

### Question 2

In [ ]:
count_tweets = df_unique['user_id'].groupby(by = df_unique['user_id']).count()

tweeets = np.sort(count_tweets.values)
number_tweets = tweeets[-5:]

max_users_ids = []
for i in range(0, 5):
    max_users_ids.append(count_tweets[count_tweets == number_tweets[i]].index[0])

df_q2 = df_unique[['user_id','user_name']]
df_q2 = df_q2.drop_duplicates(subset='user_name')

users_names = []
for each in max_users_ids:
    users_names.append(df_q2['user_name'][df_q2['user_id'] == each])


### Question 3

In [ ]:

user_ids = df['user_id']
tweet_ids = df['tweet_id']

df11 = pd.DataFrame({'mentions_users_ids' :mentions_users_ids, 'user_id': user_ids, 'tweet_ids': tweet_ids})
df11 = df11.dropna(subset='mentions_users_ids')

In [ ]:
how_many_users_mentions = []
for each in df['mentions_users_ids'].values:
    if len(each) > 1:
        for i in range(0, len(each)):
            how_many_users_mentions.append(each[i])
    else:
        how_many_users_mentions.append(each[0])
        
how_many_users_mentions_count = dict(Counter(how_many_users_mentions))
print(sorted(how_many_users_mentions_count.items(), key=lambda x: x[1], reverse=True))

### Question 4

In [ ]:
df.dropna(subset=['mentions_users_ids', 'country_code'], inplace=True)

country_codes = ['GB', 'IE', 'ES', 'FR']
df_GB = df.loc[df['country_code'] == 'GB']
df_IE = df.loc[df['country_code'] == 'IE']
df_FR = df.loc[df['country_code'] == 'FR']
df_ES = df.loc[df['country_code'] == 'ES']

In [ ]:
users_mentions_list_GB = []
for each in df_GB['mentions_users_ids'].values:
    if len(each) > 1:
        for i in range(0, len(each)):
            users_mentions_list_GB.append(int(each[i]))
    else:
        users_mentions_list_GB.append(int(each[0]))

users_mentions_list_IE = []
for each in df_IE['mentions_users_ids'].values:
    if len(each) > 1:
        for i in range(0, len(each)):
            users_mentions_list_IE.append(int(each[i]))
    else:
        users_mentions_list_IE.append(int(each[0]))

users_mentions_list_FR = []
for each in df_FR['mentions_users_ids'].values:
    if len(each) > 1:
        for i in range(0, len(each)):
            users_mentions_list_FR.append(int(each[i]))
    else:
        users_mentions_list_FR.append(int(each[0]))

users_mentions_list_ES = []
for each in df_ES['mentions_users_ids'].values:
    if len(each) > 1:
        for i in range(0, len(each)):
            users_mentions_list_ES.append(int(each[i]))
    else:
        users_mentions_list_ES.append(int(each[0]))

In [ ]:
df.loc[df['user_id'].isin(users_mentions_list_GB)].loc[df['country_code'].isin(country_codes)].groupby(by='country_code').count()

In [ ]:
df.loc[df['user_id'].isin(users_mentions_list_IE)].loc[df['country_code'].isin(country_codes)].groupby(by='country_code').count()

In [ ]:
df.loc[df['user_id'].isin(users_mentions_list_FR)].loc[df['country_code'].isin(country_codes)].groupby(by='country_code').count()

In [ ]:
df.loc[df['user_id'].isin(users_mentions_list_ES)].loc[df['country_code'].isin(country_codes)].groupby(by='country_code').count()

# Part 3

In [ ]:
part3_cols = ['tweet_id', 'country_code', 'coordinate_lon', 'coordinate_lat']
df = pd.read_csv(csv_file, usecols = part3_cols)

#Remove duplicates
df.drop_duplicates(inplace=True)
df.dropna(how='all', axis=0, inplace=True)

### Question 1

In [ ]:
df.dropna(subset='coordinate_lon', inplace=True)

gdf = geopandas.GeoDataFrame(df, geometry=geopandas.points_from_xy(df.coordinate_lon, df.coordinate_lat))
gdf.reset_index(drop=True, inplace=True)
gdf.set_crs(crs=4326, epsg=4326, inplace=True)

world_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world_map = world_map[['name', 'geometry']]
world_map.to_crs(crs=4326, epsg=4326, inplace=True)

join_left_df = gpd.sjoin(gdf, world_map, how="left").groupby(by = 'name').count().reset_index()

final = pd.merge(join_left_df, world_map, on='name')
final.drop(['country_code', 'coordinate_lon', 'coordinate_lat', 'geometry_x', 'index_right'], axis = 1, inplace = True)
final.rename({'tweet_id':'count', 'geometry_y': 'geometry'}, axis = 1, inplace = True)

gdf_final = gpd.GeoDataFrame(final, crs="EPSG:4326", geometry='geometry')

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
gdf_final.plot(ax=ax, column='count', cmap='YlOrRd', legend=False)
plt.xlim(-24.5, 69.1)
plt.ylim(34.8, 81.9)
plt.title('Number of tweets per country in June 2022', fontsize = 20)

cax = fig.add_axes([1, 0.1, 0.03, 0.8])
vmin = gdf_final['count'].min()
vmax = gdf_final['count'].max()
sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbr = fig.colorbar(sm, cax=cax)
cbr.ax.tick_params(labelsize=20) 
plt.tight_layout()

### Question 4

In [ ]:
part34_cols = ['tweet_id', 'country_code']
df = pd.read_csv(csv_file, usecols = part34_cols)

#Remove duplicates
df.drop_duplicates(inplace=True)
df.dropna(how='all', axis=0, inplace=True)

world_map = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
# world_map = world_map[['name', 'geometry']]
world_map.to_crs(crs=4326, epsg=4326, inplace=True)

In [ ]:
df = pd.read_csv(r'D:\Documentos\Jorge\University\Master\Tweets\Electricity_use_per_person.csv')
df = df[['Country', '2011']]

df_joined = df.set_index('Country').join(world_map.set_index('name'))
df_joined = gpd.GeoDataFrame(df_joined, crs="EPSG:4326", geometry='geometry')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 7))
df_joined.plot(ax=ax, column='2011', cmap='YlOrRd', legend=False)
plt.xlim(-20, 40)
plt.ylim(34, 73)
plt.title('Electricity use per capita in 2011 (kW/h)', fontsize = 20)

cax = fig.add_axes([1, 0.1, 0.03, 0.8])
vmin = df_joined['2011'].min()
vmax = df_joined['2011'].max()
sm = plt.cm.ScalarMappable(cmap='YlOrRd', norm=plt.Normalize(vmin=vmin, vmax=vmax))
cbr = fig.colorbar(sm, cax=cax)
cbr.ax.tick_params(labelsize=20) 
plt.tight_layout()

### Question 3

In [ ]:
part33_cols = ['tweet_id', 'coordinate_lon', 'point1_lon', 'point1_lat', 'point2_lon', 'point2_lat', 'point3_lon', 'point3_lat',]
df = pd.read_csv(csv_file, usecols = part33_cols)

#Remove duplicates
df.drop_duplicates(inplace=True)
df.dropna(how='all', axis=0, inplace=True)

In [ ]:
df1 = df[df['coordinate_lon'].isnull()]
df1 = df1.dropna(subset=['point1_lon'])
df1 = df1[['point1_lon', 'point1_lat', 'point3_lon', 'point3_lat']]
df1 = df1.reset_index(drop=True)

df_point1 = gpd.points_from_xy(df1.point1_lon, df1.point1_lat)
df_point3 = gpd.points_from_xy(df1.point3_lon, df1.point3_lat)
dist = df_point1.distance(df_point3)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
plt.hist(dist, bins = 100, histtype = 'step', cumulative = True);
plt.title('CDF plot of the bounding box diagonals', fontsize = 15)

# Part 4


In [ ]:
part4_cols = ['tweet_id', 'timestamp', 'full_text', 'country_code']
df = pd.read_csv(csv_file, usecols = part4_cols)

#Remove duplicates
df.drop_duplicates(inplace=True)
df.dropna(how='all', axis=0, inplace=True)

### Question 1

In [ ]:
time_series_df = df.copy()
time_series_df.dropna(how='all', axis=0, inplace=True)
time_series_df.reset_index(drop=True, inplace=True)

In [ ]:
index_time = []
for i in range(0, len(time_series_df['timestamp'])):
    timestamp = datetime.date.fromtimestamp(int(time_series_df['timestamp'][i]) / 1000)
    index_time.append(timestamp)
    

In [ ]:
time_series_df.insert(1, 'index_times', index_time)
time_series_df.drop('timestamp', axis=1, inplace=True)
time_series_df.set_index(time_series_df['index_times'], drop=True, inplace = True)
time_series_df.drop('index_times', axis=1, inplace=True)
time_series_df = time_series_df[~time_series_df['country_code'].isnull()]

In [ ]:
df_IE = time_series_df.loc[time_series_df['country_code'] == 'IE']

df_FR = time_series_df.loc[time_series_df['country_code'] == 'FR']

df_UK = time_series_df.loc[time_series_df['country_code'] == 'GB']

In [ ]:
df_IE_day = df_IE.loc[df_IE.index == datetime.date(2022, 6, 26)]
df_FR_day = df_FR.loc[df_FR.index == datetime.date(2022, 6, 19)]
df_UK_day = df_UK.loc[df_UK.index == datetime.date(2022, 6, 24)]

In [ ]:
hashtags_IE = df_IE_day['full_text'].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_IE.columns = ['hashtag','occurences']

hashtags_IE[:10].plot(kind='bar',y='occurences',x='hashtag')
plt.title('Top 10 Hashtags for Ireland on the 26-06-2022', fontsize=22);

In [ ]:
hashtags_FR = df_FR_day['full_text'].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_FR.columns = ['hashtag','occurences']

hashtags_FR[:10].plot(kind='bar',y='occurences',x='hashtag')
plt.suptitle('Top 10 Hashtags for France on the 19-06-2022', fontsize=22);

In [ ]:
hashtags_UK_1 = df_UK_day['full_text'][0:33299].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_UK_1.columns = ['hashtag','occurences']

hashtags_UK_2 = df_UK_day['full_text'][33299:66597].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_UK_2.columns = ['hashtag','occurences']

hashtags_UK_3 = df_UK_day['full_text'][66597:99896].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_UK_3.columns = ['hashtag','occurences']

hashtags_UK_4 = df_UK_day['full_text'][99896:].apply(lambda x: pd.value_counts(re.findall('(#\w+)', x.lower() )))\
                     .sum(axis=0)\
                     .to_frame()\
                     .reset_index()\
                     .sort_values(by=0,ascending=False)
hashtags_UK_4.columns = ['hashtag','occurences']

hashtags_UK = pd.concat([hashtags_UK_1, hashtags_UK_2, hashtags_UK_3, hashtags_UK_4]).groupby(['hashtag']).sum().reset_index()

hashtags_UK.sort_values(by=['occurences'], ascending=False, inplace=True)

hashtags_UK[:10].plot(kind='bar',y='occurences',x='hashtag')
plt.suptitle('Top 10 Hashtags for Great Britain on the 24-06-2022', fontsize=22);

In [ ]:
from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')
additional  = ['rt','rts','retweet']
swords_IE = set().union(stopwords.words('english'),additional)
swords_FR = set().union(stopwords.words('french'),additional) 
swords_UK = set().union(stopwords.words('english'),additional)

In [ ]:
swords_FR.add('c\'est')
swords_FR.add('ça')
swords_FR.add('c’est')
swords_FR.add('j\'ai')
swords_FR.add(r'j\'ai')
swords_FR.add('l\'')
swords_FR.add('d\'')
swords_FR.add('j\'')
swords_FR.add('l’')
swords_FR.add('j’')
swords_FR.add('d’')
swords_FR.add('n’')
swords_FR.add(r'j\'')


In [ ]:
from ast import literal_eval

textIE = []
for each in df_IE_day['full_text']:
    textIE.append(literal_eval(each))
df_IE_use = pd.DataFrame(textIE, columns=['text'])

textFR = []
for each in df_FR_day['full_text']:
    textFR.append(literal_eval(each))
df_FR_use = pd.DataFrame(textFR, columns=['text'])

textUK = []
for each in df_UK_day['full_text']:
    textUK.append(literal_eval(each))
df_UK_use = pd.DataFrame(textUK, columns=['text'])

In [ ]:
df_IE_use['processed_text'] = df_IE_use['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .str.replace('amp',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords_IE])

In [ ]:
df_FR_use['processed_text'] = df_FR_use['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('/^[a-zA-ZÀ-Ÿ]*$/',' ')\
          .str.replace(' +',' ')\
          .str.replace('amp',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords_FR])

In [ ]:
df_UK_use['processed_text'] = df_UK_use['text'].str.lower()\
          .str.replace('(@[a-z0-9]+)\w+',' ')\
          .str.replace('(http\S+)', ' ')\
          .str.replace('([^0-9a-z \t])',' ')\
          .str.replace(' +',' ')\
          .str.replace('amp',' ')\
          .apply(lambda x: [i for i in x.split() if not i in swords_UK])

In [ ]:
df_IE.dropna(inplace=True)
df_IE.reset_index(drop=True, inplace=True)
df_FR.dropna(inplace=True)
df_FR.reset_index(drop=True, inplace=True)
df_UK.dropna(inplace=True)
df_UK.reset_index(drop=True, inplace=True)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
# #Ireland
bigstring = df_IE_use['processed_text'].apply(lambda x: ' '.join(x)).str.cat(sep=' ')
plt.figure(figsize=(12,12))
wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          collocations=False,
                          width=1200,
                          height=1000
                         ).generate(bigstring)
plt.axis('off')
plt.title('Ireland Word Cloud', fontsize=20)
plt.imshow(wordcloud)

#France
bigstring = df_FR_use['processed_text'].apply(lambda x: ' '.join(x)).str.cat(sep=' ')
plt.figure(figsize=(12,12))
wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          collocations=False,
                          width=1200,
                          height=1000
                         ).generate(bigstring)
plt.axis('off')
plt.title('France Word Cloud', fontsize=20)
plt.imshow(wordcloud)

#United Kingdom
bigstring = df_UK_use['processed_text'].apply(lambda x: ' '.join(x)).str.cat(sep=' ')
plt.figure(figsize=(12,12))
wordcloud = WordCloud(stopwords=STOPWORDS,
                          background_color='white',
                          collocations=False,
                          width=1200,
                          height=1000
                         ).generate(bigstring)
plt.axis('off')
plt.title('Great Britain Word Cloud', fontsize=20)
plt.imshow(wordcloud)

In [ ]:
def f(x):
     return Series(dict(Number_of_tweets = x['tweet_id'].count(), ))

daily_count = time_series_df.groupby(time_series_df.index.date).apply(f)
print('There are', len(daily_count), 'days in the dataset')